In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import cv2
import os
from tqdm import *


In [2]:
img_list_path = "C:\\Users\\amith\\Documents\\Datasets\\10 - Land Segmentation using U-Net Architecture"

img_list = os.listdir(img_list_path)

terrain_img_list = list(map(lambda x: x if "_t" in x else None,img_list))
while None in terrain_img_list:
    terrain_img_list.remove(None)
terrain_img_list.sort()

height_img_list = list(map(lambda x: x if "_h" in x else None,img_list))
while None in height_img_list:
    height_img_list.remove(None)
height_img_list.sort()

segmented_img_list = list(map(lambda x: x if "_i2" in x else None,img_list))
while None in segmented_img_list:
    segmented_img_list.remove(None)
segmented_img_list.sort()

print(len(terrain_img_list))
print(len(height_img_list))
print(len(segmented_img_list))


5000
5000
5000


In [3]:
print(terrain_img_list[0:5])
print(height_img_list[0:5])
print(segmented_img_list[0:5])


['0001_t.png', '0002_t.png', '0003_t.png', '0004_t.png', '0005_t.png']
['0001_h.png', '0002_h.png', '0003_h.png', '0004_h.png', '0005_h.png']
['0001_i2.png', '0002_i2.png', '0003_i2.png', '0004_i2.png', '0005_i2.png']


In [5]:
X = []
y = []
for i in tqdm(range(len(terrain_img_list))):

    terrain_img = cv2.imread(filename=img_list_path + "\\" + terrain_img_list[i])
    segmented_img = cv2.imread(filename=img_list_path + "\\" + segmented_img_list[i])

    terrain_img_resized = cv2.resize(terrain_img,(128,128))
    segmented_img_resized = cv2.resize(segmented_img,(128,128))

    terrain_img_scaled = terrain_img_resized / 255.0
    segmented_img_scaled = segmented_img_resized / 255.0

    X.append(terrain_img_scaled)
    y.append(segmented_img_scaled)
X = np.array(X)
y = np.array(y)


100%|██████████| 5000/5000 [01:09<00:00, 71.97it/s] 


In [22]:
img_indices = np.array(range(len(X)))
np.random.shuffle(img_indices)

val_test_threshold = int(0.8 * len(img_indices))
train_val_threshold = int(0.8 * val_test_threshold)

train_indices = img_indices[:train_val_threshold]
val_indices = img_indices[train_val_threshold:val_test_threshold]
test_indices = img_indices[val_test_threshold:]

X_train = X[train_indices]
X_val = X[val_indices]
X_test = X[test_indices]

y_train = y[train_indices]
y_val = y[val_indices]
y_test = y[test_indices]


In [29]:
128 * 128 * 3


49152

In [31]:
model = tf.keras.models.Sequential(layers=[
    
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.UpSampling2D(size=(2,2)),

    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.UpSampling2D(size=(2,2)),

    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.UpSampling2D(size=(2,2)),

    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(128,128,3)),
    tf.keras.layers.UpSampling2D(size=(2,2)),
])


c:\Users\amith\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


KeyboardInterrupt: 

In [28]:
help(tf.keras.layers.MaxPooling2D)


Help on class MaxPooling2D in module keras.src.layers.pooling.max_pooling2d:

class MaxPooling2D(keras.src.layers.pooling.base_pooling.BasePooling)
 |  MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None, name=None, **kwargs)
 |  
 |  Max pooling operation for 2D spatial data.
 |  
 |  Downsamples the input along its spatial dimensions (height and width)
 |  by taking the maximum value over an input window
 |  (of size defined by `pool_size`) for each channel of the input.
 |  The window is shifted by `strides` along each dimension.
 |  
 |  The resulting output when using the `"valid"` padding option has a spatial
 |  shape (number of rows or columns) of:
 |  `output_shape = math.floor((input_shape - pool_size) / strides) + 1`
 |  (when `input_shape >= pool_size`)
 |  
 |  The resulting output shape when using the `"same"` padding option is:
 |  `output_shape = math.floor((input_shape - 1) / strides) + 1`
 |  
 |  Args:
 |      pool_size: int or tuple of 2 i